In [1]:
import os
import sys
sys.path.append(os.path.join('..', 'common'))

from datatypes import KnowledgeBase, Literal, Atom, Top, Bot, Normally, Formula
from util import materialized, print_knowledge_base, entails
from justifications import justifications, separate
from ranked_models import statement_ranking

In [2]:
bot = Bot()            # Falsum
top = Top()            # Verum
m = Literal(Atom('m')) # mamalian red blood cells
v = Literal(Atom('v')) # vertebrate red blood cells
a = Literal(Atom('a')) # avian red blood cells
c = Literal(Atom('c')) # cell membrane
n = Literal(Atom('n')) # nucleus
s = Literal(Atom('s')) # mammalian sickle cells
b = Literal(Atom('b')) # bioconcave shape

In [3]:
def basic_relevance_closure(knowledge_base: KnowledgeBase, statement: Normally) -> bool:
    rank = statement_ranking(knowledge_base)
    i = 0
    T, D = separate(knowledge_base)
    Rel = {j for justification in justifications(T, D, statement.left) for j in justification}
    Irr = D - Rel
    R_ = materialized(Rel)
    while R_ and entails(materialized(T) | materialized(Irr) | R_, -statement.left):
        R_ = R_ - materialized(rank[i] & Rel)
        i += 1
    return entails(materialized(T) | materialized(Irr) | R_, statement.materialize())

In [5]:
K = {-(m >> v) / bot, -(a >> v) / bot, v / c, v / n, m / -n}
T, D = separate(K)
print_knowledge_base(K)

{ m → v, m |~ ¬n, a → v, v |~ c, v |~ n }


In [6]:
justifications(T, D, Formula(a))

set()

In [7]:
basic_relevance_closure(K, a/c)

True

In [8]:
justifications(T, D, Formula(m))

{frozenset({m |~ ¬n, v |~ n})}

In [9]:
basic_relevance_closure(K, m/c)

True

In [13]:
K={-(m >> v) / bot, -(a >> v) / bot, v / c, v / n, m / -n, -(s >> m) / bot, m / b, s / -b}
T, D = separate(K)
print_knowledge_base(K)

{ m |~ b, m → v, m |~ ¬n, a → v, v |~ c, v |~ n, s |~ ¬b, s → m }


In [14]:
justifications(T, D, Formula(s))

{frozenset({m |~ b, s |~ ¬b}), frozenset({m |~ ¬n, v |~ n})}

In [15]:
basic_relevance_closure(K, s / -n)

False